In [ ]:
from zumi.zumi import Zumi
import math 
import time

# CONSTANTS
POWER = 40
zumi = Zumi()

L_THRES = None
R_THRES = None 


def setIRthresValues():
    global L_THRES, R_THRES
    left_list  = []
    right_list = []
    
    for i in range(200):
        ir_readings = zumi.get_all_IR_data()
        left_list.append( ir_readings[5])
        right_list.append(ir_readings[0])
    
    L_THRES = (max(left_list)+min(left_list))/ 2
    R_THRES = (max(right_list)+min(right_list))/ 2
    
    #L_THRES = 10+min(left_list)
    #R_THRES = 10+min(right_list)


def isDetectingObject( ir_readings ):
    global L_THRES, R_THRES
    
    front_right_ir = ir_readings[0]
    front_left_ir  = ir_readings[5]
    
    if front_left_ir <= L_THRES or front_right_ir <= R_THRES:
        return True
    else:
        return False


setIRthresValues()

In [ ]:
#RESETS 
zumi.mpu.calibrate_MPU()
zumi.reset_gyro()  # make sure to reset the gyro since we are using go_straight()


# set the total time
total_time = 2

# re-set start time & time_start
time_start   = time.time()
time_elapsed = 0
stop_time = 0
flag = False

# direction 
heading = 0

try: 
    # GOAL ".forward" and move to ".go_straight"
    # so that we are able to read IR data while moving Zumi 
    while time_elapsed  <  total_time+stop_time:

        if isDetectingObject( zumi.get_all_IR_data() ):
            if flag == False:
                ti   = time.time()
            flag = True
            print("stopping")
            zumi.stop()
        else:

            if flag == True:
                tf = time.time() 
                print( "adding time"  )
                stop_time += tf - ti 
                flag = False 

            print("moving")
            time_elapsed = time.time()-time_start
            zumi.go_straight(POWER, heading)

    print("total stop time: ", stop_time)

finally:
    zumi.stop()